In [ ]:
# Copyright (c) 2024, Edward Jakunskas, Department of Electronic Engineering, Maynooth University
#
# Maintainer: Edward Jakunskas (edward.jakunskas@mu.ie)
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
#    list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice, this
#    list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.


In [175]:
import os
import pandas as pd
import numpy as np

#base path where all files are stored and licor file prefix
base_path = '/workspace/Data/GH4-Completed'
file_prefix = 'Auto gsw+F_'  #Prefix for the licor measurements files

#List of daily directories
daily_dirs = ['2024-08-29-Morning', '2024-08-29-Evening',
              '2024-09-02-Evening', '2024-09-03-Morning',
              '2024-09-03-Evening', '2024-09-04-Morning']

#relevant licro metrics to extract
columns_of_interest = ['gsw', 'gtw', 'VPleaf', 'VPDleaf', 'H2O_leaf', 'Fs', "Fm'", 'abs', 'TRANS', 'Tleaf']

#use this fucntion to find licor excel files with a certain rpefix
def find_file_with_prefix(directory, prefix):
    for file in os.listdir(directory):
        if file.startswith(prefix) and file.endswith('.xlsx'):
            return os.path.join(directory, file)
    return None

#Function to load, process Li-COR data, handle merged cells, filter columns, bascially cleaning up to calcualte correlations easily
def process_individual_licor_data(base_path, daily_dirs, file_prefix, columns_of_interest):
    for date_dir in daily_dirs:
        directory_path = os.path.join(base_path, date_dir)
        
        #find the file that starts with the licor prefix
        file_path = find_file_with_prefix(directory_path, file_prefix)
        
        if file_path and os.path.exists(file_path):
            licor_data = pd.read_excel(file_path, header=1)  #Uses the second row as headers

            #rename the first column to pot id manually since merged cells affect header
            licor_data.rename(columns={licor_data.columns[0]: 'Pot ID'}, inplace=True)

            #ffill down the pot id column where there are merged cells
            licor_data['Pot ID'].fillna(method='ffill', inplace=True)
            
            #select cleaned pot id's and columns of interest 
            licor_data = licor_data[['Pot ID'] + columns_of_interest]
            
            #get average reading for each pot id
            licor_data_avg = licor_data.groupby('Pot ID').mean().reset_index()
            
            #Save the processed data to the respective directory
            save_path = os.path.join(directory_path, 'processed_LiCOR_data_cleaned.xlsx')
            licor_data_avg.to_excel(save_path, index=False)
            print(f"Processed and saved data to: {save_path}")
        else:
            print(f"File not found with prefix '{file_prefix}' in {directory_path}")

#Process each Li-COR file individually and save the results to the respective directories
process_individual_licor_data(base_path, daily_dirs, file_prefix, columns_of_interest)


Processed and saved data to: /workspace/Data/GH4-Completed/2024-08-29-Morning/processed_LiCOR_data_cleaned.xlsx
Processed and saved data to: /workspace/Data/GH4-Completed/2024-08-29-Evening/processed_LiCOR_data_cleaned.xlsx
Processed and saved data to: /workspace/Data/GH4-Completed/2024-09-02-Evening/processed_LiCOR_data_cleaned.xlsx
Processed and saved data to: /workspace/Data/GH4-Completed/2024-09-03-Morning/processed_LiCOR_data_cleaned.xlsx
Processed and saved data to: /workspace/Data/GH4-Completed/2024-09-03-Evening/processed_LiCOR_data_cleaned.xlsx
Processed and saved data to: /workspace/Data/GH4-Completed/2024-09-04-Morning/processed_LiCOR_data_cleaned.xlsx


In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

#base folder and daily directories we'll loop through
base_path = '/workspace/Data/GH4-Completed'

daily_dirs = ['2024-08-29-Morning',
              '2024-09-03-Morning',
              '2024-09-04-Morning']

li_cor_file_name = 'processed_LiCOR_data_cleaned.xlsx'
camera_file_name = 'merged_file_with_pages.xlsx'

#lists to collect combined camera and li-cor data
all_median_camera_data = []
all_li_cor_data = []

#load licor data for a specific date
def get_licor_data(base_path, date_dir, li_cor_file_name):
    file_path = os.path.join(base_path, date_dir, li_cor_file_name)
    
    if os.path.isfile(file_path):
        df = pd.read_excel(file_path)
        df['Pot ID'] = df['Pot ID'].astype(str) + f"_{date_dir}"  #combining pot ID and date to have unique correlations for each pot/date
        return df
    else:
        print(f"File '{li_cor_file_name}' not found in {file_path}")
        return None


#loading camera data for a specific sheet on a specific date
def load_camera_data(base_path, date_dir, sheet_name):
    file_path = os.path.join(base_path, date_dir, camera_file_name)
    
    if os.path.isfile(file_path):
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        df.rename(columns={df.columns[0]: 'Pot ID'}, inplace=True)  #manually naming first column to Pot ID as this wasn't done in previous code, can ignore if fixed
        return df
    else:
        print(f"Camera file '{camera_file_name}' not found in {file_path}")
        return None


#matchign and sortign the pots betwen the licor and camera sheets
def match_and_sort_pots(camera_sheets, li_cor_data, date_dir):
    camera_filtered_sheets = []
    
    # Iterate through each camera sheet and filter by matching Pot IDs
    for camera_sheet in camera_sheets:
        camera_sheet['Pot ID'] = camera_sheet['Pot ID'].astype(str) + f"_{date_dir}" #licor date appended while reading licor file
        
        # find pot IDs that also exist in licor data, then sort by p[ot ID so it lines up with licor data
        camera_filtered = camera_sheet[camera_sheet['Pot ID'].isin(li_cor_data['Pot ID'])].sort_values('Pot ID')
        camera_filtered_sheets.append(camera_filtered)

    # remove licor data that isn' tin matching camera data, sort by pot ID also
    all_pot_ids = pd.concat([camera_sheet['Pot ID'] for camera_sheet in camera_sheets])
    li_cor_filtered = li_cor_data[li_cor_data['Pot ID'].isin(all_pot_ids)].sort_values('Pot ID')
    #returning list of matched and sorted camera and licor data
    return camera_filtered_sheets, li_cor_filtered


#computing the median value of each pot angle as a simple way to remove outliers
def median_camera_sheets(camera_filtered_sheets):
    concatenated_df = pd.concat([sheet.select_dtypes(include=[np.number]) for sheet in camera_filtered_sheets],
                                keys=range(len(camera_filtered_sheets)))

    median_df = concatenated_df.groupby(level=1).median()
    return median_df


# Find Pearson correlations between camera and Li-COR data, tested spearmans' ranked correlations and found the correlation is more linear
def find_correlations_pearson(median_camera, li_cor_data):
    correlations = {}
    
    #select only the numeric columns, excluding 'Pot ID'
    camera_numeric_columns = median_camera.select_dtypes(include=[np.number]).columns
    licor_numeric_columns = li_cor_data.select_dtypes(include=[np.number]).columns

    for camera_col in camera_numeric_columns:
        for licor_col in licor_numeric_columns:
            corr_value, p_value = pearsonr(median_camera[camera_col], li_cor_data[licor_col])
            if not np.isnan(corr_value):
                correlations.setdefault(licor_col, []).append((camera_col, corr_value, p_value))
    
    return correlations

#helper function
def process_correlation(camera_col, licor_col, corr_value, p_value, num_tests, bonferroni_threshold, significant_camera_metrics, normalized_correlations_list):
    # Adjust the p-value with Bonferroni correction
    adjusted_p_value = min(p_value * num_tests, 1)  # Ensure p-value doesn't exceed 1
    is_significant = adjusted_p_value < 0.05
    
    # Append significant metrics and correlations
    if is_significant:
        significant_camera_metrics.add(camera_col)
    
    # Save correlation data to the list
    normalized_correlations_list.append({
        'LiCOR Metric': licor_col,
        'Camera Metric': camera_col,
        'Correlation': corr_value,
        'Original P-value': p_value,
        'Adjusted P-value': adjusted_p_value,
        'Bonferroni Threshold': bonferroni_threshold,
        'Significant': 'Yes' if is_significant else 'No'
    })
    
def plot_and_save(camera_col, licor_col, corr_value, p_value, median_camera, li_cor_data, output_dir, plot_index):
    plt.figure(figsize=(6, 4))
    plt.scatter(median_camera[camera_col], li_cor_data[licor_col], alpha=0.5)
    plt.title(f'{camera_col} vs {licor_col}\nCorrelation: {corr_value:.2f}, p-value: {p_value:.3g}')
    plt.xlabel(camera_col)
    plt.ylabel(licor_col)
    plt.grid(True)
    plot_file_name = f'{camera_col}_vs_{licor_col}_corr_{plot_index + 1}.png'.replace('/', '_')
    plt.savefig(os.path.join(output_dir, plot_file_name))
    plt.close()
    
def plot_significant_correlations_per_licor(correlations, median_camera, li_cor_data, n=5, num_tests=1, output_dir='correlation_plots', output_file='normalized_significant_correlations.csv'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    bonferroni_threshold = 0.05 / num_tests
    normalized_correlations_list = []
    significant_camera_metrics = set()
    total_significant_correlations = 0
    total_correlations = 0

    for licor_col, correlation_data in correlations.items():
        sorted_correlations = sorted(correlation_data, key=lambda x: abs(x[1]), reverse=True)
        print(f"Top {n} correlations for Li-COR Metric: {licor_col}")

        for i, (camera_col, corr_value, p_value) in enumerate(sorted_correlations):
            process_correlation(camera_col, licor_col, corr_value, p_value, num_tests, bonferroni_threshold, significant_camera_metrics, normalized_correlations_list)
            total_correlations += 1
            if i < n:
                print(f"{i+1}. {camera_col} vs {licor_col} | Correlation: {corr_value}, p-value: {p_value}")
                plot_and_save(camera_col, licor_col, corr_value, p_value, median_camera, li_cor_data, output_dir, i)

    normalized_correlations_df = pd.DataFrame(normalized_correlations_list)
    normalized_correlations_df.to_csv(output_file, index=False)

    print(f"Total significant correlations: {len([d for d in normalized_correlations_list if d['Significant'] == 'Yes'])}")
    print(f"Total number of camera metrics with at least 1 significant correlation: {len(significant_camera_metrics)}")
    print(f"Total correlations: {total_correlations}")


# Main loop: process each date
for date_dir in daily_dirs:
    li_cor_data = get_licor_data(base_path, date_dir, li_cor_file_name)
    
    if li_cor_data is not None:
        camera_sheets = [load_camera_data(base_path, date_dir, sheet) for sheet in 
                         ['Horizontal-Right', 'Horizontal-Left', 'Angled-45-Right', 'Angled-45-Left']]

        if any(sheet is None for sheet in camera_sheets):
            print(f"Skipping {date_dir} due to missing camera sheets.") #check if anything missing
            continue
        
        camera_filtered_sheets, li_cor_filtered = match_and_sort_pots(camera_sheets, li_cor_data, date_dir)
        median_camera = median_camera_sheets(camera_filtered_sheets)
        
        all_median_camera_data.append(median_camera)
        all_li_cor_data.append(li_cor_filtered)

#Combine all daily data
combined_median_camera = pd.concat(all_median_camera_data, ignore_index=True)
combined_li_cor_data = pd.concat(all_li_cor_data, ignore_index=True)

#Find correlations
correlations = find_correlations_pearson(combined_median_camera, combined_li_cor_data)

#Plot the top correlations
plot_significant_correlations_per_licor(
    correlations, 
    combined_median_camera, 
    combined_li_cor_data, 
    n=5,  # Show top 5 correlations in plots
    num_tests=117,  # 117 tests, this is a bug that is present in paper, should be 117*8, which results in 42 significant correlations instead of 111, but bonferonni seems too strict for this data, so this needs to be resolved shortly regardless
    output_dir='correlation_plots_cleaned_v2', 
    output_file='normalized_significant_correlations_cleaned_v2.csv'
)

/tmp/ipykernel_8564/2114992210.py:85: NearConstantInputWarning: An input array is nearly constant; the computed correlation coefficient may be inaccurate.
  corr_value, p_value = pearsonr(median_camera[camera_col], li_cor_data[licor_col])
/tmp/ipykernel_8564/2114992210.py:85: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr_value, p_value = pearsonr(median_camera[camera_col], li_cor_data[licor_col])


Top 5 correlations for Li-COR Metric: gsw
1. HI Mean vs gsw | Correlation: -0.38872745184204616, p-value: 0.00015291273656362853
2. NGRDI Min vs gsw | Correlation: -0.3743179789308916, p-value: 0.00027849472208608055
3. HI Q3 vs gsw | Correlation: -0.3737042024344305, p-value: 0.00028552402457794054
4. HI Median vs gsw | Correlation: -0.37036391888143816, p-value: 0.0003267266514710615
5. HI Mode vs gsw | Correlation: -0.3671110982675979, p-value: 0.0003720411491392104
Top 5 correlations for Li-COR Metric: gtw
1. HI Mean vs gtw | Correlation: -0.3892020933220807, p-value: 0.00014985154337750483
2. HI Q3 vs gtw | Correlation: -0.3740846063030888, p-value: 0.00028114843795281973
3. HI Median vs gtw | Correlation: -0.3709741783049636, p-value: 0.000318813063462927
4. HI Mode vs gtw | Correlation: -0.3685490153530486, p-value: 0.0003513414487242904
5. NGRDI Min vs gtw | Correlation: -0.3655897097595036, p-value: 0.00039515762398214583
Top 5 correlations for Li-COR Metric: VPleaf
1. GREEN R